In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# --- Load Dataset ---
data = pd.read_csv("heart.csv")
print("Dataset shape:", data.shape)
print(data.head())

Dataset shape: (1025, 14)
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  


In [4]:
# --- Separate features and target ---
X = data.drop('target', axis=1)
y = data['target']

In [5]:
# One-hot encode categorical columns if any (optional but good practice)
X = pd.get_dummies(X, drop_first=True)

In [6]:
# --- Split into training and testing sets ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
# --- Define a simple neural network model ---
def build_model(input_dim):
    model = keras.Sequential([
        layers.Dense(16, activation='relu', input_shape=(input_dim,)),
        layers.Dense(8, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
# --- Helper function to train and evaluate ---
def train_and_evaluate(X_train, X_test, y_train, y_test, desc):
    model = build_model(X_train.shape[1])
    history = model.fit(X_train, y_train, epochs=50, batch_size=16,
                        validation_split=0.2, verbose=0)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    print(f"\n--- {desc} ---")
    print(f"Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}")
    return history

In [9]:
# --- 1. Raw Data ---
hist_raw = train_and_evaluate(X_train, X_test, y_train, y_test, "Raw (Unscaled)")

C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step 

--- Raw (Unscaled) ---
Accuracy: 0.771, Precision: 0.764, Recall: 0.786


In [10]:
# --- 2. MinMax Scaled ---
scaler_mm = MinMaxScaler()
X_train_mm = scaler_mm.fit_transform(X_train)
X_test_mm = scaler_mm.transform(X_test)
hist_mm = train_and_evaluate(X_train_mm, X_test_mm, y_train, y_test, "MinMax Scaled")

C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 

--- MinMax Scaled ---
Accuracy: 0.839, Precision: 0.812, Recall: 0.883


In [11]:
# --- 3. Standard Scaled ---
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train)
X_test_std = scaler_std.transform(X_test)
hist_std = train_and_evaluate(X_train_std, X_test_std, y_train, y_test, "Standard Scaled")

C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step 

--- Standard Scaled ---
Accuracy: 0.873, Precision: 0.860, Recall: 0.893
